<a href="https://colab.research.google.com/github/bb426/textbooks/blob/master/5_deep_learning_for_computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras import layers, models

Using TensorFlow backend.
